In [1]:
import os
import time
import json
import wfdb
import paho.mqtt.client as mqtt
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

def publish_ecg(record_path, broker_host="localhost", broker_port=1883, base_topic="ecg"):
    # Extract device_id
    person_folder = os.path.basename(os.path.dirname(record_path))  # Person_01
    person_num = person_folder.split("_")[1]                        # 01
    device_id = f"D_{person_num}"

    meta_topic = f"{base_topic}/meta/{device_id}"
    data_topic = f"{base_topic}/data/{device_id}"

    # Load ECG record
    record = wfdb.rdrecord(record_path)

    fs = record.fs
    channels = record.sig_name
    signals = record.p_signal
    n_samples = record.sig_len

    start_time_iso = datetime.utcnow().isoformat()

    # Metadata message
    metadata_msg = {
        "device_id": device_id,
        "record_id": record.record_name,
        "start_time": start_time_iso,
        "fs": fs,
        "channels": channels,
        "metadata": {
            "age": 25,
            "sex": "male",
            "lead_units": record.units,
            "adc_gain": record.adc_gain
        }
    }

    # MQTT client
    client = mqtt.Client(client_id=device_id)
    client.connect(broker_host, broker_port, keepalive=60)
    client.loop_start()

    # Publish metadata once
    client.publish(meta_topic, payload=json.dumps(metadata_msg), qos=1, retain=True)
    print(f"{device_id}: Published metadata")

    # Wall-clock locked streaming
    start_wall = time.time()
    for i in range(n_samples):
        msg = {
            "device_id": device_id,
            "record_id": record.record_name,
            "start_time": start_time_iso,
            "ts": round(i / fs, 6),
            "signals": {
                "ecg_i": float(signals[i, 0]),
                "ecg_i_filtered": float(signals[i, 1])
            }
        }

        client.publish(data_topic, payload=json.dumps(msg), qos=1, retain=False)

        target_time = start_wall + (i + 1) / fs
        sleep_time = target_time - time.time()
        if sleep_time > 0:
            time.sleep(sleep_time)

    duration = time.time() - start_wall
    print(f"{device_id}: Simulation finished in {duration:.3f} seconds")

    client.loop_stop()
    client.disconnect()

# ---------- RUN 20 PARALLEL SIMULATIONS ----------
def get_record_files(base_path="/home/ubuntu2020/dataset/ecg-id-database-1.0.0"):
    files = []
    for i in range(1, 21):
        person = f"Person_{i:02d}"           # Person_01 ... Person_20
        record = os.path.join(base_path, person, "rec_1")
        files.append(record)
    return files

record_files = get_record_files()

t0 = time.time()
with ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(publish_ecg, record_files)
t1 = time.time()

print("Total simulation duration for 20 parallel devices:", round(t1 - t0, 3), "seconds")


/tmp/ipykernel_677240/1637042959.py:48: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=device_id)


Published metadata
Simulation finished in 20.000 seconds
Total simulation duration: 20.049 seconds
